# Scraper le prix neuf des voitures

## 1. Importer la liste des modèles


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import sys
import os
import glob
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.scraping.scraping import init_driver
from src.scraping.scraping import accept_popup_general

In [ ]:
data_path = "../data/raw_data/autohero.csv"
df = pd.read_csv(data_path, sep=",", encoding="utf-8")

In [3]:
df.head()

,scraped_at,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,nb_place,couleur,sellerie,classe_emission,emission_CO2,crit_air,usage_commerciale_anterieure,url_annonce
0,2025-04-09,Ford Fiesta,1.0 EcoBoost ST-Line X,13 190 €,15.05.2020,69 301 km,Essence,Boite de vitesse manuelle,95 CV / 70 kW,3,Citadine,5.0,5.0,Gris,Tissu (Sellerie d'origine),EURO 6,NaN,Crit'Air 1,NaN,https://www.autohero.com/fr/ford-fiesta/id/516...
1,2025-04-09,Toyota ProAce,Combi Long 1.5 D-4D Dynamic,23 990 €,29.04.2021,71 887 km,Diesel,Boite de vitesse manuelle,120 CV / 88 kW,2,Monospace,4.0,9.0,Gris,Tissu (Sellerie d'origine),EURO 6,170 g/km,Crit'Air 2,Oui,https://www.autohero.com/fr/toyota-pro-ace/id/...
2,2025-04-09,Mercedes-Benz GLA,250 e AMG Line 8G-DCT,32 490 €,23.10.2020,59 649 km,Hybride,Double embrayage / DCT,218 CV / 160 kW,2,SUV,5.0,5.0,Gris,Mi-cuir (Sellerie d'origine),EURO 6,32 g/km,Crit'Air 1,Oui,https://www.autohero.com/fr/mercedes-benz-gla/...
3,2025-04-09,BMW X1,sDrive18i xLine DKG7,27 890 €,05.05.2021,37 869 km,Essence,Double embrayage / DCT,136 CV / 100 kW,2,SUV,5.0,5.0,Noir,Mi-cuir (Sellerie d'origine),EURO 6,148 g/km,Crit'Air 1,Non,https://www.autohero.com/fr/bmw-x-1/id/490f203...
4,2025-04-09,Peugeot 3008,1.5 Blue-HDi Crossway EAT8,19 090 €,31.12.2019,58 958 km,Diesel,Boite de vitesse automatique,130 CV / 96 kW,3,SUV,5.0,5.0,Blanc,Mi-cuir (Sellerie d'origine),EURO 6,98 g/km,Crit'Air 2,Oui,https://www.autohero.com/fr/peugeot-3008/id/df...


In [4]:
df.shape

(2352, 20)

Il y a 2352 lignes (annonces) et 22 colonnes

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   scraped_at                    2352 non-null   object 
 1   modele                        2342 non-null   object 
 2   finition                      2342 non-null   object 
 3   prix                          2342 non-null   object 
 4   annee_mise_en_circulation     2342 non-null   object 
 5   kilometrage                   2342 non-null   object 
 6   carburant                     2342 non-null   object 
 7   transmission                  2342 non-null   object 
 8   puissance                     2342 non-null   object 
 9   nb_ancien_proprietaire        2342 non-null   object 
 10  classe_vehicule               2342 non-null   object 
 11  nb_porte                      2342 non-null   float64
 12  nb_place                      2342 non-null   float64
 13  cou

In [6]:
# Extraire l'année de mise en circulation et la convertir en int
df["annee"] = (df["annee_mise_en_circulation"]
               .str.extract(r'(\d{4})')[0]
               .astype('Int64'))                                  

In [7]:
# Extraire la marque de la voiture et l'insérer à côté de la colonne "modele"
# en utilisant la première partie de la chaîne avant l'espace
df.insert(df.columns.get_loc("modele"), "marque", df["modele"].str.split(" ").str[0])

## 2. Base des modèles à récupérer le prix neuf  
On a besoin de:
- Nom de modèle  
- Finition  
- Année  

In [8]:
# Liste des Modèle, finition et année
df_model = df[["marque", "modele", "finition", "annee"]].drop_duplicates()
df_model = df_model.sort_values(by=["marque", "modele", "finition", "annee"])
df_model.reset_index(drop=True, inplace=True)
df_model.to_csv("../data/processed_data/modeles_voitures.csv", index=False)

In [9]:
# df_model = pd.read_csv('../data/processed_data/modeles_voitures.csv')

In [10]:
df_model.head()

,marque,modele,finition,annee
0,Abarth,Abarth 124 Spider,1.4 Turbo Turismo BVA,2019
1,Abarth,Abarth 500,1.4 Turbo T-Jet 595,2020
2,Abarth,Abarth 500,1.4 Turbo T-Jet 595,2022
3,Abarth,Abarth 500,1.4 Turbo T-Jet 595 Pista,2018
4,Abarth,Abarth 500,1.4 Turbo T-Jet 595 Turismo,2020


In [11]:
# Check missing values
df_model.isnull().sum()

marque      1
modele      1
finition    1
annee       1
dtype: int64

In [12]:
# Supprimer missing values
df_model = df_model.dropna()

In [13]:
df_model["annee"] = df_model["annee"].astype(int)

In [14]:
# Dimension
df_model.shape

(1613, 4)

In [15]:
# Liste des modèles et années
df = df_model[["marque", "modele", "annee"]].drop_duplicates()
df = df.sort_values(by=["marque", "modele", "annee"])
df.reset_index(drop=True, inplace=True)

In [16]:
df.head()

,marque,modele,annee
0,Abarth,Abarth 124 Spider,2019
1,Abarth,Abarth 500,2018
2,Abarth,Abarth 500,2019
3,Abarth,Abarth 500,2020
4,Abarth,Abarth 500,2022


In [17]:
print(df["marque"].unique())
print(len(df["marque"].unique()))

['Abarth' 'Alfa' 'Audi' 'BMW' 'Citroen' 'Cupra' 'DS' 'Dacia' 'Fiat' 'Ford'
 'Honda' 'Hyundai' 'Infiniti' 'Jaguar' 'Jeep' 'Kia' 'Land' 'Lexus' 'MG'
 'MINI' 'Mazda' 'Mercedes-Benz' 'Mitsubishi' 'Nissan' 'Opel' 'Peugeot'
 'Renault' 'Seat' 'Skoda' 'Smart' 'Suzuki' 'Toyota' 'Volkswagen' 'Volvo']
34


In [18]:
# Rename "marque" to match with the right name used in the caradisiac website
dict_name_marque = {'Abarth' : 'Abarth', 'Alfa' : 'Alfa Romeo', 'Audi' : 'Audi', 
                    'BMW' : 'BMW', 
                    'Citroen' : 'Citroen', 'Cupra' : 'Cupra', 
                    'DS' : 'DS', 'Dacia' : 'Dacia',
                    'Fiat' : 'Fiat', 'Ford' : 'Ford', 
                    'Honda' : 'Honda', 'Hyundai' : 'Hyundai', 
                    'Infiniti' : 'Infiniti', 
                    'Jaguar' : 'Jaguar', 'Jeep' : 'Jeep',
                    'Kia' : 'Kia', 
                    'Land' : 'Land Rover', 'Lexus' : 'Lexus', 
                    'MG' : "MG", 'MINI' : 'MINI', 'Mazda' : 'Mazda', 'Mercedes-Benz' : 'Mercedes', 'Mitsubishi' : 'Mitsubishi', 
                    'Nissan' : 'Nissan', 
                    'Opel' : 'Opel', 
                    'Peugeot' : 'Peugeot', 
                    'Renault' : 'Renault', 
                    'Seat' : 'Seat', 'Skoda' : 'Skoda', 'Smart' : 'Smart', 'Suzuki' : 'Suzuki', 
                    'Toyota' : 'Toyota', 
                    'Volkswagen' : 'Volkswagen', 'Volvo' : 'Volvo'
                    }

df['marque'] = df['marque'].replace(dict_name_marque)
print(df["marque"].unique())
print(len(df["marque"].unique()))

['Abarth' 'Alfa Romeo' 'Audi' 'BMW' 'Citroen' 'Cupra' 'DS' 'Dacia' 'Fiat'
 'Ford' 'Honda' 'Hyundai' 'Infiniti' 'Jaguar' 'Jeep' 'Kia' 'Land Rover'
 'Lexus' 'MG' 'MINI' 'Mazda' 'Mercedes' 'Mitsubishi' 'Nissan' 'Opel'
 'Peugeot' 'Renault' 'Seat' 'Skoda' 'Smart' 'Suzuki' 'Toyota' 'Volkswagen'
 'Volvo']
34


In [19]:
freq_marque = df["marque"].value_counts().reset_index().sort_values(by='marque')
freq_marque

,marque,count
25,Abarth,7
21,Alfa Romeo,10
5,Audi,45
8,BMW,35
1,Citroen,53
28,Cupra,4
19,DS,13
15,Dacia,19
12,Fiat,29
6,Ford,37


## 2. Essai avec request et BeautifulSoup

In [20]:
# Function to scrape car catalogue prices from Caradisiac
def clean_model_name(model):
    # Remove special characters and normalize spaces
    model_v2 = re.sub(r'[^\w\s]', '', model).strip().lower()
    model_v3 = model_v2.replace(' ', '-')
    return model_v3

def scrape_caradisiac_price(modele, annee):
    base_url = "https://www.caradisiac.com/fiches-techniques/modele--"
    
    # Build search URL
    clean_model = clean_model_name(modele)
    search_url = f"{base_url}{clean_model}/{str(annee)}"
    
    print(f"Search url original pattern is: {search_url}")
    
    try:
        # First attempt with original pattern
        time.sleep(5)
        response = requests.get(search_url)
        #print(f"Code Status: {response.status_code}")
        if response.status_code != 200:
            # Try alternative URL patterns
            alternative_models = [
                clean_model.replace('_', ''), # Remove "_"
                '-'.join([clean_model.replace('_', ''), "2e", "generation"])  # join with version
            ] + ['-'.join([clean_model.replace('_', ''), str(i)]) for i in range(1,6)]
            
            #print(f"Alternative urls: {alternative_models}")
            
            for alt_model in alternative_models:
                time.sleep(5)
                alt_url = f"{base_url}{alt_model}/{str(annee)}"
                response = requests.get(alt_url)
                if response.status_code == 200:
                    search_url = alt_url
                    break
        # Add delay to be respectful to the server
        time.sleep(5)
        
        # Make request
        print(f"L'url final est: {search_url}")
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all car versions
        versions = soup.find_all('table', class_='listingTab')

        # Extract all rows from the table
        table = versions[0]  # Since versions contains only one table
        rows = table.find_all('tr')

        # Initialize lists to store data
        data = []
        headers = []

        # Get headers from first row
        header_row = rows[0]
        headers = [th.get_text(strip=True) for th in header_row.find_all('th')]

        # Get data from remaining rows
        for row in rows[1:]:
            cols = row.find_all('td')
            row_data = [col.get_text(strip=True) for col in cols]
            if row_data:  # Only add non-empty rows
                data.append(row_data)
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        df['url'] = search_url
        return df    
    except Exception as e:
        print(f"Error scraping {modele}: {str(e)}")
        return None

In [21]:
# # Create an empty DataFrame to store all results
# df_all = pd.DataFrame()

# # Add progress tracking
# total_models = len(df)
# print(f"Starting to scrape {total_models} models...")

# for idx, row in df.iterrows():
#     try:
#         print(f"Processing {idx+1}/{total_models}: {row['modele']} ({row['annee']})")
#         df_version = scrape_caradisiac_price(row['modele'], row['annee'])
        
#         if df_version is not None:
#             # Add model and year columns to identify the source
#             df_version.insert(0, 'source_model', row['modele'])
#             df_version.insert(1, 'source_year', row['annee'])
#             df_all = pd.concat([df_all, df_version], ignore_index=True)
            
#     except Exception as e:
#         print(f"Error processing {row['modele']} ({row['annee']}): {str(e)}")
#         continue

# print(f"\nScraping completed. Total entries collected: {len(df_all)}")

## 3. Essai avec Selenium pour retrouver les bons urls


In [22]:
def select_option_contain (select_element, partial_text):
    select = Select(select_element)

    #print(f"Partial text est: {partial_text}")
    accents = {
            'a': r'[àáâãäå]',
            'e': r'[èéêë]',
            'i': r'[ìíîï]',
            'o': r'[òóôõö]',
            'u': r'[ùúûü]',
            'c': r'[ç]',
            'n': r'[ñ]',
            'A': r'[ÀÁÂÃÄÅ]',
            'E': r'[ÈÉÊË]',
            'I': r'[ÌÍÎÏ]',
            'O': r'[ÒÓÔÕÖ]',
            'U': r'[ÙÚÛÜ]',
            'C': r'[Ç]',
            'N': r'[Ñ]',
        }
    # Remove special characters with accent
    for remplacement, pattern in accents.items():
        partial_text = re.sub(pattern, remplacement, partial_text)
    print(f"Partial text: {partial_text}")

    # First check for exact matches
    for option in select.options:
        #if option.get_attribute("value").strip() != "" and partial_text.strip().lower() == option.text.strip().lower():
        if (option.get_attribute("value").strip() != "" and 
            partial_text.lower() == option.text.lower()
            ):
            select.select_by_visible_text(option.text)
            print(f"Found exact match: {option.text}")
            option_select = "Very Exact match"
            return option.text, option_select
            break
        
    # Check for exact matches after removing space and "-"
    for option in select.options:
        if (option.get_attribute("value").strip() != "" and 
            re.sub(r"[\s\-]", "", partial_text.strip().lower()) == re.sub(r"[\s\-]", "", option.text.strip().lower())
            ): # remove any space and "-"
            select.select_by_visible_text(option.text)
            print(f"Found exact match: {option.text}")
            option_select = "Exact match"
            return option.text, option_select
            break
             
    # If no exact match found, check for partial matches
    best_match = None
    max_common_chars = 0
    
    for option in select.options:
        if option.get_attribute("value").strip() != "":
            option_text = re.sub(r"[\s\-]", "", option.text.strip().lower()) # remove any space
            partial_text_lower = re.sub(r"[\s\-]", "", partial_text.strip().lower()) # remove any space
            
            # Count common characters
            common_chars = sum(1 for c in option_text if c in partial_text_lower)
            
            # Check if this is a partial match and has more common characters
            if ((partial_text_lower in option_text) or (option_text in partial_text_lower)) and common_chars > max_common_chars:
                max_common_chars = common_chars
                best_match = option
    # Retain the best partial match = the option that has more common characters with the partial text
    if best_match is not None:
        select.select_by_visible_text(best_match.text)
        print(f"Found best partial match: {best_match.text}")
        option_select = "Partial match"
        return best_match.text, option_select


In [23]:
def clean_name(model):
    # Remove special characters and normalize spaces
    accents = {
            'a': r'[àáâãäå]',
            'e': r'[èéêë]',
            'i': r'[ìíîï]',
            'o': r'[òóôõö]',
            'u': r'[ùúûü]',
            'c': r'[ç]',
            'n': r'[ñ]',
            'A': r'[ÀÁÂÃÄÅ]',
            'E': r'[ÈÉÊË]',
            'I': r'[ÌÍÎÏ]',
            'O': r'[ÒÓÔÕÖ]',
            'U': r'[ÙÚÛÜ]',
            'C': r'[Ç]',
            'N': r'[Ñ]',
        }

    for remplacement, pattern in accents.items():
        model = re.sub(pattern, remplacement, model)
    model = re.sub(r'[^\w\s\-!]', '', model).strip().lower()

    model = model.replace(' ', '-')
    
    return model

In [24]:
# list_models = pd.unique(df['modele'])
# print(list_models)
# print([i for i in list_models if "!" in i ])

# list_models_cleaned = [clean_name(model) for model in list_models]
# print(list_models_cleaned)

# list_models_test = [any(char in i for char in 'éèêëàâäôöûüçñ') for i in list_models_cleaned]
# print([i for i in list_models_test if i == True])

In [25]:
def collect_prix_neuf (url, marque, modele, annee, wait_sec=10):
    '''
    Guessing the convenient urls
    Arguments:

    '''

    try:
        # Initialiser le driver
        driver = init_driver()
        driver.get(url)
        time.sleep(2)
        
        # Accept pop-up
        accept_popup_general(driver, "#didomi-notice-agree-button")
        time.sleep(5)
        driver.execute_script("document.location.reload()")
        wait = WebDriverWait(driver, wait_sec)

        df_all_versions = pd.DataFrame()
        
        # Sélectionner les options de dropdowns: Marques, Gammes, Annees, Modeles

        # Marque: if contains marque
        time.sleep(2)
        #marque_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'brands')))
        marque_dropdown = driver.find_element(By.ID, 'brands')
        #option_marque =  Select(marque_dropdown).select_by_visible_text(marque)
        option_marque, match_type_marque = select_option_contain (marque_dropdown, marque)
        print(f"Marque: {option_marque}")

        # Gammes: if contains modele
        #modele_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'models')))
        time.sleep(2)
        modele_dropdown = driver.find_element(By.ID, 'models')
        #print(f"Modèle initial est: {modele}")
        option_modele, match_type_modele = select_option_contain (modele_dropdown, modele)
        print(f"Modèle: {option_modele}")
        
        # Annees = annee
        #annee_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'year')))
        time.sleep(2)
        annee_dropdown = driver.find_element(By.ID, 'year')
        option_year, match_type_year = select_option_contain(annee_dropdown, str(annee))
        #print(f"Liste année: {[option.text for option in Select(annee_dropdown).options]}")
        #print(f"Année choisie: {option_year}")
        
        # Modeles: extract all (for i in range modeles.options)
        #modelscomm_dropdown = wait.until(EC.presence_of_element_located((By.ID, 'modelscomm')))
        time.sleep(2)
        modelscomm_dropdown = driver.find_element(By.ID, 'modelscomm')
        select_modelscomm = Select(modelscomm_dropdown)

        # Exclure version vide
        versions = [option.text for option in select_modelscomm.options if option.get_attribute("value").strip() != ""]
        print(f"Liste des versions: {versions}")
       
        if versions:  # This checks if the list is not empty
            for version in versions:
                data = []
                #select_modelscomm.select_by_visible_text(version.text)
                option_marque_cleaned = clean_name(option_marque)
                option_version = clean_name(version)
                
                search_url_version = f"{url}/modele--{option_marque_cleaned}-{option_version}/{option_year}"
                print(f"Extraire la fiche technique de: {version} - année {option_year}")
                print(f"Search url est : {search_url_version}")

                try:
                    driver.get(search_url_version)
                    time.sleep(2)
                    
                    # Récupérer le tableau "listingTab"
                    table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'table.listingTab')))

                    # Get all rows including header row
                    rows = table.find_elements(By.TAG_NAME, 'tr')

                    # Get headers from first row
                    headers = rows[0].find_elements(By.TAG_NAME, 'th')
                    header_texts = [header.text for header in headers]

                    # Get data from remaining rows
                    for row in rows[1:]:
                        cells = row.find_elements(By.TAG_NAME, 'td')
                        row_data = [cell.text for cell in cells]
                        if row_data:
                            data.append(row_data)

                    # Sauvegarder le tableau dans un dataframe
                    df = pd.DataFrame(data, columns=header_texts)

                    # Création des colonnes pour identifier le type de matching
                    df['url'] = driver.current_url
                    df['option_marque_select'] = option_marque
                    df['option_modele_select'] = option_modele
                    df['option_year_select'] = option_year
                    df['match_type_marque'] = match_type_marque
                    df['match_type_modele'] = match_type_modele
                    df['match_type_year'] = match_type_year

                    #print(f"URL used is: {driver.current_url}")

                    #print(df.head())
                    
                    df_all_versions = pd.concat([df_all_versions, df], ignore_index=True)
                    #driver.close()
                    #print(f"df all version est: {df_all_versions.head()}")
                except: # If error then continue with a new iteration
                    continue
        # Close the driver    
        driver.quit() 
        return df_all_versions
    except Exception as e:
        print (f"URL is not correct")
        print (f"Error is: {e}")
        driver.quit()
        return None

In [26]:
# url = "https://www.caradisiac.com/fiches-techniques"
# marque = "BMW"
# modele = "BMW Série 1"
# annee = "2021"
# df_test = collect_prix_neuf (url, marque, modele, annee)

In [27]:
def scraping_prix_neuf (df, csv_path_ok, csv_path_ko, wait_time = 10):
    # Create an empty DataFrame to store all results
    df_all = pd.DataFrame()

    df_error = pd.DataFrame()

    # Add progress tracking
    total_models = len(df)
    print(f"⏳ Starting to scrape {total_models} models & years...")

    # url
    url = "https://www.caradisiac.com/fiches-techniques"

    for idx, row in df.iterrows():
        try:
            print(f"--> 🔄Processing {idx+1}/{total_models}: {row['modele']} ({row['annee']})")
            df_version = collect_prix_neuf (url, row['marque'], row['modele'], row['annee'], wait_time)
            
            if df_version is not None:
                # Add model and year columns to identify the source
                df_version.insert(0, 'source_model', row['modele'])
                df_version.insert(1, 'source_year', row['annee'])
                df_all = pd.concat([df_all, df_version], ignore_index=True)
                print(f"✅Successfully collected for {row['modele']} ({row['annee']})")
            else:
                print(f"❌Error processing {row['modele']} ({row['annee']})")
                df_error = pd.concat([df_error, pd.DataFrame({'modele': [row['modele']], 'annee': [row['annee']]})], ignore_index=True)    
        except Exception as e:
            print(f"❌Error processing {row['modele']} ({row['annee']}): {str(e)}")
            continue

    print(f"\n 🆗 Scraping completed. Total entries collected: {len(df_all)}")
    # Exporter les données dans un fichier CSV
    df_all.to_csv(csv_path_ok, index=False, encoding='utf-8-sig')
    print(f"\n ⚠️ Total models not found: {len(df_error)}")
    df_error.to_csv(csv_path_ko, index=False, encoding='utf-8-sig')
    return None

In [28]:
# Fonction pour diviser le df original en plusieurs parties => pour lancer le scraping en plusieurs patchs
def split_dataframe_by_size(df, nb_lines, csv_root):
    list_dfs = [df[i:i+nb_lines] for i in range(0, len(df), nb_lines)]

    somme_originale = len(df)
    somme_lignes = sum(len(split) for split in list_dfs)

    print(f"Somme des lignes de la base brute = {somme_originale}")
    print(f"Somme des lignes de tous les splits dataframe = {somme_lignes}")
    print(f'# Data split: {len(list_dfs)}')

    # Save each split dataframe to CSV
    for i, split_df in enumerate(list_dfs):
        csv_path = f'{csv_root}/split_car_models_{i+1}.csv'
        split_df.to_csv(csv_path, index=False)
        print(f'Split {i+1} saved to {csv_path}')
    
    return list_dfs

In [29]:
list_df_split = split_dataframe_by_size(df, 50, "../data/raw_data")

Somme des lignes de la base brute = 752
Somme des lignes de tous les splits dataframe = 752
# Data split: 16
Split 1 saved to ../data/raw_data/split_car_models_1.csv
Split 2 saved to ../data/raw_data/split_car_models_2.csv
Split 3 saved to ../data/raw_data/split_car_models_3.csv
Split 4 saved to ../data/raw_data/split_car_models_4.csv
Split 5 saved to ../data/raw_data/split_car_models_5.csv
Split 6 saved to ../data/raw_data/split_car_models_6.csv
Split 7 saved to ../data/raw_data/split_car_models_7.csv
Split 8 saved to ../data/raw_data/split_car_models_8.csv
Split 9 saved to ../data/raw_data/split_car_models_9.csv
Split 10 saved to ../data/raw_data/split_car_models_10.csv
Split 11 saved to ../data/raw_data/split_car_models_11.csv
Split 12 saved to ../data/raw_data/split_car_models_12.csv
Split 13 saved to ../data/raw_data/split_car_models_13.csv
Split 14 saved to ../data/raw_data/split_car_models_14.csv
Split 15 saved to ../data/raw_data/split_car_models_15.csv
Split 16 saved to ../dat

### First attempt

In [30]:
# Process by pack
path = "../data/raw_data"
pattern = "split_car_models_*.csv"
files = glob.glob(os.path.join(path, pattern))
nb_split_df = len(files)

for i, path_split in enumerate(files):
    csv_path_ok = '../data/processed_data/prix_neuf_voitures_pack' + str(i+1) + '.csv'
    
    # Only execute if output file doesn't exist
    if not os.path.exists(csv_path_ok):
        data = pd.read_csv(path_split)
        csv_path_ko = '../data/processed_data/prix_neuf_voitures_pack' + str(i+1) + '_error.csv'

        # Run scraping
        scraping_prix_neuf(data, csv_path_ok, csv_path_ko)
    else:
        print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_pack1.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack2.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack3.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack4.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack5.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack6.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack7.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack8.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack9.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack10.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack11.csv already exists, skipping...
File ../data/processed_data/prix_neuf_voitures_pack12.csv alrea

In [31]:
# Liste des modèles & années à collecter
df['modele_annee'] = df['modele'] + ' ' + df['annee'].astype(str)
unique_modele_annee = sorted(pd.unique(df["modele_annee"]))
print(f"{len(unique_modele_annee)} modèles et années à collecter: \n {unique_modele_annee}")

# Liste des modèles & années qui n'ont pas été collectés
path = "../data/processed_data"
pattern = "prix_neuf_voitures_pack*.csv"
files = [file for file in glob.glob(os.path.join(path, pattern)) if "error" not in file]
print(files)
df_prix_neuf = pd.DataFrame()
for file in files:
    df_ = pd.read_csv(file)
    df_prix_neuf = pd.concat([df_prix_neuf, df_], ignore_index=True)
df_prix_neuf['modele_annee'] = df_prix_neuf['source_model'] + ' ' + df_prix_neuf['source_year'].astype(str)
unique_modele_annee_collected = sorted(pd.unique(df_prix_neuf["modele_annee"]))
print(f"{len(unique_modele_annee_collected)} modèles et années collectés: \n {unique_modele_annee_collected}")

# Liste des modèles & années qui ont été collectés (sans doublons)
df_prix_neuf['modele_annee_option'] = df_prix_neuf['option_marque_select'] +  ' ' + df_prix_neuf['option_modele_select'] + ' ' + df_prix_neuf['option_year_select'].astype(str)
unique_modele_annee_option = sorted(pd.unique(df_prix_neuf["modele_annee_option"]))
print(f"{len(unique_modele_annee_option)} modèles et années sélectionnés: \n {unique_modele_annee_option}")

unique_modele_annee_final = df_prix_neuf[['source_model', 'source_year', 'modele_annee_option']].drop_duplicates()
unique_modele_annee_final['duplicate_source_models'] = unique_modele_annee_final.groupby('modele_annee_option')['source_model'].transform(lambda x: '#'.join(x) if len(x) > 1 else None)
df_doublons = unique_modele_annee_final[['modele_annee_option', 'duplicate_source_models']].drop_duplicates()
df_doublons = df_doublons[df_doublons['duplicate_source_models'].notnull()]
df_doublons

752 modèles et années à collecter: 
 ['Abarth 124 Spider 2019', 'Abarth 500 2018', 'Abarth 500 2019', 'Abarth 500 2020', 'Abarth 500 2022', 'Abarth 595 2017', 'Abarth 595C 2020', 'Alfa Romeo Giulia 2020', 'Alfa Romeo Giulia 2021', 'Alfa Romeo Giulietta 2017', 'Alfa Romeo Giulietta 2018', 'Alfa Romeo Giulietta 2019', 'Alfa Romeo Giulietta 2021', 'Alfa Romeo Stelvio 2017', 'Alfa Romeo Stelvio 2018', 'Alfa Romeo Tonale 2023', 'Alfa Romeo Tonale 2024', 'Audi A1 2017', 'Audi A1 2018', 'Audi A1 2019', 'Audi A1 2020', 'Audi A1 2021', 'Audi A1 2023', 'Audi A1 citycarver 2021', 'Audi A3 2017', 'Audi A3 2018', 'Audi A3 2019', 'Audi A3 2020', 'Audi A3 2021', 'Audi A3 2022', 'Audi A3 2023', 'Audi A3 Limousine 2017', 'Audi A3 Limousine 2019', 'Audi A3 Limousine 2021', 'Audi A4 2018', 'Audi A4 2019', 'Audi A4 2020', 'Audi A4 2021', 'Audi A4 2022', 'Audi A5 Sportback 2020', 'Audi A5 Sportback 2022', 'Audi A6 2018', 'Audi Q2 2017', 'Audi Q2 2018', 'Audi Q2 2019', 'Audi Q2 2020', 'Audi Q2 2022', 'Audi 

650 modèles et années collectés: 
 ['Abarth 124 Spider 2019', 'Abarth 500 2018', 'Abarth 500 2019', 'Abarth 500 2020', 'Abarth 500 2022', 'Alfa Romeo Giulia 2020', 'Alfa Romeo Giulia 2021', 'Alfa Romeo Giulietta 2017', 'Alfa Romeo Giulietta 2018', 'Alfa Romeo Giulietta 2019', 'Alfa Romeo Giulietta 2021', 'Alfa Romeo Stelvio 2017', 'Alfa Romeo Stelvio 2018', 'Alfa Romeo Tonale 2023', 'Alfa Romeo Tonale 2024', 'Audi A1 2017', 'Audi A1 2018', 'Audi A1 2019', 'Audi A1 2020', 'Audi A1 2021', 'Audi A1 citycarver 2021', 'Audi A3 2017', 'Audi A3 2018', 'Audi A3 2019', 'Audi A3 2021', 'Audi A3 2022', 'Audi A3 2023', 'Audi A3 Limousine 2017', 'Audi A3 Limousine 2019', 'Audi A3 Limousine 2021', 'Audi A4 2018', 'Audi A4 2019', 'Audi A4 2020', 'Audi A4 2021', 'Audi A4 2022', 'Audi A5 Sportback 2020', 'Audi A5 Sportback 2022', 'Audi A6 2018', 'Audi Q2 2017', 'Audi Q2 2018', 'Audi Q2 2019', 'Audi Q2 2020', 'Audi Q2 2022', 'Audi Q2 2023', 'Audi Q3 2017', 'Audi Q3 2018', 'Audi Q3 2019', 'Audi Q3 2020',

,modele_annee_option,duplicate_source_models
1393,AUDI A1 2021,Audi A1#Audi A1 citycarver
1497,AUDI A3 2017,Audi A3#Audi A3 Limousine
2462,AUDI A3 2019,Audi A3#Audi A3 Limousine
2737,AUDI A3 2021,Audi A3#Audi A3 Limousine
8787,FIAT 500 2017,Fiat 500#Fiat 500C#Fiat 500X
8981,FIAT 500 2018,Fiat 500#Fiat 500C#Fiat 500X
9194,FIAT 500 2019,Fiat 500#Fiat 500C#Fiat 500X
16198,HYUNDAI I30 2019,Hyundai i30#Hyundai i30 Fastback
23573,MERCEDES CLASSE A 2019,Mercedes-Benz Classe A#Mercedes-Benz Classe A ...
23668,MERCEDES CLASSE A 2020,Mercedes-Benz Classe A#Mercedes-Benz Classe A ...


### Second attempt

In [32]:
# Liste des modèles & années restant à collecter
remaining_to_collect = set(unique_modele_annee) - set(unique_modele_annee_collected)
print(len(remaining_to_collect))
print(sorted(remaining_to_collect))

102
['Abarth 595 2017', 'Abarth 595C 2020', 'Audi A1 2023', 'Audi A3 2020', 'BMW Série 4 2019', 'BMW Série 4 2022', 'Citroen C4 Grand Picasso 2017', 'Citroen C4 Grand Picasso 2019', 'Citroen C4 Picasso 2019', 'Citroen DS3 2018', 'Citroen DS3 Cabrio 2017', 'Citroen DS3 Cabrio 2019', 'Citroen DS4 2018', 'Citroen DS4 Crossback 2018', 'Citroen DS5 2017', 'Citroen DS5 2018', 'DS Automobiles DS3 2019', 'DS Automobiles DS3 2022', 'DS Automobiles DS3 2023', 'DS Automobiles DS3 Crossback 2019', 'DS Automobiles DS3 Crossback 2020', 'DS Automobiles DS3 Crossback 2021', 'DS Automobiles DS3 Crossback 2022', 'DS Automobiles DS4 2021', 'DS Automobiles DS4 2022', 'DS Automobiles DS7 2022', 'DS Automobiles DS7 Crossback 2018', 'DS Automobiles DS7 Crossback 2020', 'DS Automobiles DS7 Crossback 2021', 'Dacia Dokker 2019', 'Dacia Duster 2018', 'Fiat 500 2020', 'Fiat 500 2021', 'Fiat 500 2022', 'Fiat 500C 2020', 'Fiat 500C 2021', 'Fiat 500L 2020', 'Fiat 500X 2020', 'Fiat 500X 2021', 'Ford EcoSport 2021', '

In [33]:
# Dataframe des modèles restant à collecter
df2 = df[df['modele_annee'].isin(remaining_to_collect)]
df2 = df2.reset_index(drop=True)

In [34]:
df2.head()

,marque,modele,annee,modele_annee
0,Abarth,Abarth 595,2017,Abarth 595 2017
1,Abarth,Abarth 595C,2020,Abarth 595C 2020
2,Audi,Audi A1,2023,Audi A1 2023
3,Audi,Audi A3,2020,Audi A3 2020
4,BMW,BMW Série 4,2019,BMW Série 4 2019


In [35]:
# Il y avait peut-être des erreurs lors du 1er scraping à cause du serveur trop chargé
# 2è essaie...
csv_path_ok = '../data/processed_data/prix_neuf_voitures_essaie_2.csv'
if not os.path.exists(csv_path_ok):
    csv_path_ko = '../data/processed_data/prix_neuf_voitures_essaie_2_error.csv'
    scraping_prix_neuf (df2, csv_path_ok, csv_path_ko)
else:
    print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_essaie_2.csv already exists, skipping...


In [36]:
# Importer les résultats
df2_ok = pd.read_csv('../data/processed_data/prix_neuf_voitures_essaie_2.csv')
df2_ok.head()

,source_model,source_year,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year
0,Audi A1,2023,II ALLSTREET 25 TFSI 95 ADVANCED BVM5,5.0,Ess.,Mécanique,130 (wltp),27 040 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
1,Audi A1,2023,II ALLSTREET 25 TFSI 95 DESIGN BVM5,5.0,Ess.,Mécanique,129 (wltp),26 510 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
2,Audi A1,2023,II ALLSTREET 30 TFSI 110 ADVANCED BVM6,5.0,Ess.,Mécanique,128 (wltp),27 750 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
3,Audi A1,2023,II ALLSTREET 30 TFSI 110 ADVANCED S TRONIC 7,5.0,Ess.,Automatique,141 (wltp),29 650 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match
4,Audi A1,2023,II ALLSTREET 30 TFSI 110 AVUS BVM6,5.0,Ess.,Mécanique,128 (wltp),33 200 €,https://www.caradisiac.com/fiches-techniques/m...,AUDI,A1,2023,Exact match,Partial match,Exact match


In [37]:
# Liste des modèles qui sont collectés avec la 2è essaie
df2_ok['modele_annee'] = df2_ok['source_model'] + ' ' + df2_ok['source_year'].astype(str)
unique_modele_annee_collected_sup = sorted(pd.unique(df2_ok["modele_annee"]))
print(f"{len(unique_modele_annee_collected_sup)} modèles et années collectés avec la 2è essaie: \n {unique_modele_annee_collected_sup}")

# Liste des modèles qui ne sont toujours pas collectés avec la 2è essaie
remaining_to_collect_sup = sorted(set(remaining_to_collect) - set(unique_modele_annee_collected_sup))
print(f"{len(remaining_to_collect_sup)} modèles et années restant à collecter: \n {remaining_to_collect_sup}")

38 modèles et années collectés avec la 2è essaie: 
 ['Audi A1 2023', 'DS Automobiles DS3 2019', 'DS Automobiles DS3 2022', 'DS Automobiles DS3 2023', 'DS Automobiles DS3 Crossback 2020', 'DS Automobiles DS3 Crossback 2021', 'DS Automobiles DS3 Crossback 2022', 'DS Automobiles DS4 2021', 'DS Automobiles DS4 2022', 'DS Automobiles DS7 2022', 'DS Automobiles DS7 Crossback 2018', 'DS Automobiles DS7 Crossback 2020', 'DS Automobiles DS7 Crossback 2021', 'Dacia Dokker 2019', 'Dacia Duster 2018', 'Fiat 500L 2020', 'Fiat 500X 2020', 'Fiat 500X 2021', 'Mazda 3 2022', 'Mazda MX-5 2017', 'Mazda MX-5 2019', 'Peugeot 2008 2018', 'Renault Twingo 2017', 'Renault Twingo 2018', 'Seat Ibiza 2021', 'Seat Leon 2019', 'Skoda Kodiaq 2018', 'Skoda Kodiaq 2020', 'Skoda Kodiaq 2021', 'Toyota Auris 2017', 'Toyota Aygo 2018', 'Toyota Aygo 2019', 'Toyota Aygo X 2022', 'Toyota Aygo X 2024', 'Toyota C-HR 2021', 'Toyota Yaris 2024', 'Toyota Yaris Cross 2021', 'Toyota Yaris Cross 2022']
64 modèles et années restant à

### Third attempt

In [38]:
df3 = df[df['modele_annee'].isin(remaining_to_collect_sup)]
df3 = df3.reset_index(drop=True)
df3

,marque,modele,annee,modele_annee
0,Abarth,Abarth 595,2017,Abarth 595 2017
1,Abarth,Abarth 595C,2020,Abarth 595C 2020
2,Audi,Audi A3,2020,Audi A3 2020
3,BMW,BMW Série 4,2019,BMW Série 4 2019
4,BMW,BMW Série 4,2022,BMW Série 4 2022
...,...,...,...,...
59,Toyota,Toyota Yaris,2017,Toyota Yaris 2017
60,Toyota,Toyota Yaris,2018,Toyota Yaris 2018
61,Toyota,Toyota Yaris,2019,Toyota Yaris 2019
62,Toyota,Toyota Yaris,2020,Toyota Yaris 2020


In [39]:
# 3è essaie avec wait time plus élevé
csv_path_ok = '../data/processed_data/prix_neuf_voitures_essaie_3.csv'
if not os.path.exists(csv_path_ok):
    csv_path_ko = '../data/processed_data/prix_neuf_voitures_essaie_3_error.csv'
    scraping_prix_neuf (df3, csv_path_ok, csv_path_ko, 20)
else:
    print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_essaie_3.csv already exists, skipping...


### Fourth attempt

In [40]:
df3_ok = pd.read_csv('../data/processed_data/prix_neuf_voitures_essaie_3.csv')
# Liste des modèles qui sont collectés avec la 3è essaie
df3_ok['modele_annee'] = df3_ok['source_model'] + ' ' + df3_ok['source_year'].astype(str)
unique_modele_annee_collected_sup_2 = sorted(pd.unique(df3_ok["modele_annee"]))
print(f"{len(unique_modele_annee_collected_sup_2)} modèles et années collectés avec la 3è essaie: \n {unique_modele_annee_collected_sup_2}")

# Liste des modèles qui ne sont toujours pas collectés avec la 3è essaie
remaining_to_collect_sup_2 = sorted(set(remaining_to_collect_sup) - set(unique_modele_annee_collected_sup_2))
print(f"{len(remaining_to_collect_sup_2)} modèles et années restant à collecter: \n {remaining_to_collect_sup_2}")

27 modèles et années collectés avec la 3è essaie: 
 ['Audi A3 2020', 'BMW Série 4 2019', 'BMW Série 4 2022', 'DS Automobiles DS3 Crossback 2019', 'Fiat 500 2020', 'Fiat 500 2021', 'Fiat 500 2022', 'Fiat 500C 2020', 'Fiat 500C 2021', 'Ford EcoSport 2021', 'Ford EcoSport 2022', 'Mazda 6 2020', 'Opel Astra 2018', 'Opel Corsa 2017', 'Opel Corsa 2018', 'Opel Corsa 2019', 'Renault Mégane 2017', 'Renault Mégane 2018', 'Renault Mégane 2019', 'Renault Mégane 2020', 'Renault Mégane 2021', 'Renault Mégane 2022', 'Toyota Yaris 2017', 'Toyota Yaris 2018', 'Toyota Yaris 2019', 'Toyota Yaris 2020', 'Volkswagen Caddy 2019']
37 modèles et années restant à collecter: 
 ['Abarth 595 2017', 'Abarth 595C 2020', 'Citroen C4 Grand Picasso 2017', 'Citroen C4 Grand Picasso 2019', 'Citroen C4 Picasso 2019', 'Citroen DS3 2018', 'Citroen DS3 Cabrio 2017', 'Citroen DS3 Cabrio 2019', 'Citroen DS4 2018', 'Citroen DS4 Crossback 2018', 'Citroen DS5 2017', 'Citroen DS5 2018', 'Ford Ka+ 2018', 'Ford Tourneo 2020', "Kia 

In [41]:
df4 = df3[df3['modele_annee'].isin(remaining_to_collect_sup_2)].reset_index(drop=True)
# liste des modèles
list_modele_df4 = pd.unique(df4['modele'])
print(list_modele_df4)

['Abarth 595' 'Abarth 595C' 'Citroen C4 Grand Picasso'
 'Citroen C4 Picasso' 'Citroen DS3' 'Citroen DS3 Cabrio' 'Citroen DS4'
 'Citroen DS4 Crossback' 'Citroen DS5' 'Ford Ka+' 'Ford Tourneo'
 "Kia cee'd" "Kia pro_cee'd" 'Land Rover Evoque'
 'Mercedes-Benz Classe GLB' 'Mercedes-Benz Classe GLC'
 'Mercedes-Benz Classe GLE' 'Opel Crossland X' 'Suzuki SX4 S-Cross']


In [42]:
# Corriger manuellement le nom du modèle/marque pour qu'il soit aligné avec celui utilisé par le site
dict_model_corr = {

 'Abarth 595' : '500',
 'Abarth 595C' : '500',
 'Citroen C4 Grand Picasso' : 'C4 PICASSO',
 'Citroen C4 Picasso' : 'C4 PICASSO', 
 'Citroen DS3' : 'DS3',
 'Citroen DS3 Cabrio' : 'DS3',
 'Citroen DS4' : 'DS4',
 'Citroen DS4 Crossback' : 'DS4 CROSSBACK',
 'Citroen DS5' : 'DS5',
 'Ford Ka+' : 'KA+',
 'Ford Tourneo' : 'TOURNEO COURIER',
 "Kia cee'd" : 'CEED',
 "Kia pro_cee'd": 'PROCEED',
 'Land Rover Evoque' : 'RANGE ROVER EVOQUE',
 'Mercedes-Benz Classe GLB' : 'GLB',
 'Mercedes-Benz Classe GLC' : 'GLC',
 'Mercedes-Benz Classe GLE' : 'GLE',
 'Opel Crossland X' : 'CROSSLAND' ,
 'Suzuki SX4 S-Cross' : 'S-CROSS'
}

In [43]:
# Create columns to keep original "marque" & "modele" name
df4["marque_init"] = df4["marque"]
df4['modele_init'] = df4['modele']

# Replace the "modele" name
df4['modele'] = df4['modele'].replace(dict_model_corr)
df4.head()

,marque,modele,annee,modele_annee,marque_init,modele_init
0,Abarth,500,2017,Abarth 595 2017,Abarth,Abarth 595
1,Abarth,500,2020,Abarth 595C 2020,Abarth,Abarth 595C
2,Citroen,C4 PICASSO,2017,Citroen C4 Grand Picasso 2017,Citroen,Citroen C4 Grand Picasso
3,Citroen,C4 PICASSO,2019,Citroen C4 Grand Picasso 2019,Citroen,Citroen C4 Grand Picasso
4,Citroen,C4 PICASSO,2019,Citroen C4 Picasso 2019,Citroen,Citroen C4 Picasso


In [44]:
#df4[(df4["modele"].str.contains("DS", na=False)) & (df4["marque"].str.contains("Citroen", na=False))]

In [45]:
# Rename "marque" Citroen = DS if "modele" contains DS and "marque" contains Citroen
df4.loc[(df4["modele"].str.contains("DS", na=False)) 
        & (df4["marque"].str.contains("Citroen", na=False)), "marque"] = "DS"
df4['modele_annee_upd'] = df4['modele']+ ' ' + df4['annee'].astype(str)
list_to_collect_4th = sorted(pd.unique(df4['modele_annee_upd']))
print(f"{len(list_to_collect_4th)} à collecter: \n {list_to_collect_4th}")

36 à collecter: 
 ['500 2017', '500 2020', 'C4 PICASSO 2017', 'C4 PICASSO 2019', 'CEED 2018', 'CEED 2019', 'CEED 2021', 'CEED 2022', 'CEED 2023', 'CROSSLAND 2023', 'DS3 2017', 'DS3 2018', 'DS3 2019', 'DS4 2018', 'DS4 CROSSBACK 2018', 'DS5 2017', 'DS5 2018', 'GLB 2020', 'GLB 2021', 'GLB 2022', 'GLC 2017', 'GLC 2018', 'GLC 2019', 'GLC 2020', 'GLC 2021', 'GLE 2018', 'KA+ 2018', 'PROCEED 2017', 'PROCEED 2019', 'PROCEED 2020', 'PROCEED 2021', 'RANGE ROVER EVOQUE 2017', 'RANGE ROVER EVOQUE 2018', 'RANGE ROVER EVOQUE 2020', 'S-CROSS 2021', 'TOURNEO COURIER 2020']


In [46]:
# url = "https://www.caradisiac.com/fiches-techniques"
# marque = "KIA"
# modele = "PROCEED"
# annee = "2021"
# df_test = collect_prix_neuf (url, marque, modele, annee)

In [47]:
# 4è essaie
csv_path_ok = '../data/processed_data/prix_neuf_voitures_essaie_4.csv'
if not os.path.exists(csv_path_ok):
    csv_path_ko = '../data/processed_data/prix_neuf_voitures_essaie_4_error.csv'
    scraping_prix_neuf (df4, csv_path_ok, csv_path_ko, 10)
else:
    print(f"File {csv_path_ok} already exists, skipping...")

File ../data/processed_data/prix_neuf_voitures_essaie_4.csv already exists, skipping...


In [48]:
# Import the result from 4th attempt
df4_ok = pd.read_csv(csv_path_ok)
df4_ok

# Liste des modèles qui sont collectés avec la 4è essaie
df4_ok['modele_annee'] = df4_ok['source_model'] + ' ' + df4_ok['source_year'].astype(str)
unique_modele_annee_collected_sup_3 = sorted(pd.unique(df4_ok["modele_annee"]))
print(f"{len(unique_modele_annee_collected_sup_3)} modèles et années collectés avec la 4è essaie: \n {unique_modele_annee_collected_sup_3}")

# Liste des modèles qui ne sont toujours pas collectés avec la 3è essaie
remaining_to_collect_sup_3 = sorted(set(list_to_collect_4th) - set(unique_modele_annee_collected_sup_3))
print(f"{len(remaining_to_collect_sup_3)} modèles et années restant à collecter: \n {remaining_to_collect_sup_3}")

34 modèles et années collectés avec la 4è essaie: 
 ['500 2017', '500 2020', 'C4 PICASSO 2017', 'CEED 2018', 'CEED 2019', 'CEED 2021', 'CEED 2022', 'CEED 2023', 'CROSSLAND 2023', 'DS3 2017', 'DS3 2018', 'DS3 2019', 'DS4 2018', 'DS4 CROSSBACK 2018', 'DS5 2017', 'DS5 2018', 'GLB 2020', 'GLB 2021', 'GLB 2022', 'GLC 2017', 'GLC 2018', 'GLC 2019', 'GLC 2020', 'GLC 2021', 'GLE 2018', 'KA+ 2018', 'PROCEED 2019', 'PROCEED 2020', 'PROCEED 2021', 'RANGE ROVER EVOQUE 2017', 'RANGE ROVER EVOQUE 2018', 'RANGE ROVER EVOQUE 2020', 'S-CROSS 2021', 'TOURNEO COURIER 2020']
2 modèles et années restant à collecter: 
 ['C4 PICASSO 2019', 'PROCEED 2017']


### Final

In [49]:
# Concatenate the dataframes
df_combined = pd.concat([df_prix_neuf, df2_ok, df3_ok, df4_ok], ignore_index=True)

# Display the combined dataframe
df_combined.head()

# Remove duplicates
df_combined_nodup = df_combined.drop_duplicates(subset=['option_marque_select', 'option_modele_select', 'option_year_select', 'Versions'])
list_final = sorted(pd.unique(df_combined_nodup["modele_annee"]))

print(len(list_final))

724


In [50]:
df_combined_nodup.head()

,source_model,source_year,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_modele_select,option_year_select,match_type_marque,match_type_modele,match_type_year,modele_annee,modele_annee_option
0,Abarth 124 Spider,2019,II 1.4 TURBO 170,2.0,Ess.,Mécanique,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
1,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT,2.0,Ess.,Mécanique,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
2,Abarth 124 Spider,2019,II 1.4 TURBO 170 GT BVA,2.0,Ess.,Automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
3,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO,2.0,Ess.,Mécanique,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019
4,Abarth 124 Spider,2019,II 1.4 TURBO 170 TURISMO BVA,2.0,Ess.,Automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,124,2019,Exact match,Partial match,Exact match,Abarth 124 Spider 2019,ABARTH 124 2019


In [51]:
# Remove missing price
df_combined_nodup = df_combined_nodup[df_combined_nodup["Prix"].notna()]
df_combined_nodup['Prix'].describe()

count        52423
unique        5829
top       28 900 €
freq           129
Name: Prix, dtype: object

In [52]:
# Checking missing price is well removed
missing_price = df_combined_nodup['Prix'].isnull().sum()
print(missing_price)

0


In [53]:
# Extract the version from the URL
df_combined_nodup['Version_selected'] = df_combined_nodup['url'].str.extract(r'--(.*?)/')
df_combined_nodup['Version_selected'] = df_combined_nodup['Version_selected'].str.replace('-', ' ', regex=False)

In [54]:
df_combined_nodup['option_marque_select'] = df_combined_nodup['option_marque_select'].str.upper()
df_combined_nodup['Versions'] = df_combined_nodup['Versions'].str.upper()
df_combined_nodup.drop(columns=['source_year', 'option_modele_select',
                                'match_type_marque', 'match_type_modele', 'match_type_year',
                                'modele_annee', 'modele_annee_option',  ], inplace=True)

In [55]:
df_combined_nodup.head()

,source_model,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_year_select,Version_selected
0,Abarth 124 Spider,II 1.4 TURBO 170,2.0,Ess.,Mécanique,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
1,Abarth 124 Spider,II 1.4 TURBO 170 GT,2.0,Ess.,Mécanique,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
2,Abarth 124 Spider,II 1.4 TURBO 170 GT BVA,2.0,Ess.,Automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
3,Abarth 124 Spider,II 1.4 TURBO 170 TURISMO,2.0,Ess.,Mécanique,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
4,Abarth 124 Spider,II 1.4 TURBO 170 TURISMO BVA,2.0,Ess.,Automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider


In [56]:
# Exporter les données dans un fichier CSV
df_combined_nodup.to_csv('../data/processed_data/prix_neuf_voitures_vf.csv', index=False, encoding='utf-8-sig')
print("Data exported successfully to ../data/processed_data/prix_neuf_voitures.csv")

Data exported successfully to ../data/processed_data/prix_neuf_voitures.csv


## 4. Merging with initial data to get the listing price

In [57]:
# df_model[(df_model['modele'].str.lower().str.contains("ds", na=False)) & (df_model['marque'].str.lower().str.contains('citroen', na=False))]

In [58]:
# Rename "marque" Citroen = DS if "modele" contains DS and "marque" contains Citroen
# df_model.loc[(df_model['marque'].str.lower().str.contains('citroen', na=False)) & (df_model['modele'].str.lower().str.contains('ds', na=False)), "marque"] = "DS"

In [59]:
# Checking again, should be empty
# df_model[(df_model['modele'].str.lower().str.contains("ds", na=False)) & (df_model['marque'].str.lower().str.contains('citroen', na=False))]

In [60]:
# Some cleaning
# df_model['marque'] = df_model['marque'].str.upper()
# df_model['finition'] = df_model['finition'].str.upper()

In [61]:
# Merging df_model with df_combined_nodup to get the listing price
# df_merge_1 = df_model.merge(df_combined_nodup, how="left", left_on=['marque', 'modele', 'annee'], right_on = ['option_marque_select', 'source_model', 'option_year_select'] )

In [62]:
# Checking the merge "exactly" or "partially"
# df_merge_1['match_type'] = df_merge_1.apply(lambda row: 'Exact' if row['finition'] == row['Versions'] else 'Partial', axis=1)

In [63]:
# df_merge_1[df_merge_1['finition'].str.upper().str.contains("SPORTBACK 30 TFSI", na=False)]

In [64]:
# df_combined_nodup[(df_combined_nodup['option_marque_select'].str.upper().str.contains("AUDI", na=False))
#                   & (df_combined_nodup['source_year'].astype(str) == "2023")
#                   ]